<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung



> Prüfen wie stark der CCI gewachsen ist in den letzten 14 Tagen

# **Weitere Ideen**


> Aktien Sektoren zuordnen und den Bot nach Sektoren abfragen. Der Bot soll dann die stärkste Aktie aus dem Sektor zurückgeben. Ziemlich cool, wenn man weiß das zum Beispiel der Wasserstoffsektor gerade extrem am steigen ist.




> Sobald man sich für eine Aktie entschieden, wäre es ja auch interessant zu wissen, wann man am besten aussteigen sollte. Dann könnte man Abfragen schreiben und über den Bot eine Push rausschicken, ob man eine bestimmte Aktie, die man mit dem Bot teilt, verkaufen soll oder nicht, weil sich gerade der Kurs schlecht entwickelt.





# **Indikatoren**


> **MACD & Signallinie**: Der MACD (Moving Average Convergence/Divergence) wird in der Chartanalyse genutzt um Trends sowie Kauf-und Verkaufssignale zu erkennen. Dabei werden zwei exponentiell gewichtete gleitende Durchschnitte (26 und 12 Tage) berechnet und voneinander subtrahiert. Des Weiteren wird eine Signallinie berechnet. Hierbei handelt es sich um den 9 Tage exponentiell geglätteten Durchschnitt. Ein steigender MACD ist ein Indikator für einen Aufwärtstrend, ein sinkender MACD ein Indikator für einen Abwärtstrend. Schneidet der MACD die Signallinie von unten nach oben, so liegt ein Kaufsignal vor. Wird die Signallinie vom MACD von oben nach unten geschnitten, so liegt ein Verkaufssignal vor.



> **RSI**: Der RSI (Relative-Stärke-Index) ist ein Marktindikator. Er wird berechnet, indem der gleitende Mittelwert der Auf-und Abwärtsveränderungen für einen bestimmten Zeitraum (in der Regel 14 Tage) ermittelt wird. Der RSI kann Werte zwischen 0 und 100 einnehmen. Die Werte 30 und 70 spielen beim RSI eine besondere Rolle: Bei einem Wert unter 30 gilt der Wert als überverkauft. Dies bedeutet, dass eine Kurserholung (Steigerung) in nächster Zeit anstehen könnte. Bei einem Wert über 70 gilt die Aktie als überkauft. Dies ist ein Indiz dafür, dass der Kurs in nächster Zeit fallen könnte.




> **Bollinger Band**: Es existieren eigentlich zwei Bollinger Bänder, die gemeinsam mit dem gleitenden Mittel, dem arithmetischen Mittel der Kurse, dann das sogenannte Bollinger Band bilden. Mit Hilfe des Bollinger Bands werden der Verlauf der letzten Tage eines Wertpapieres und dessen Trend visualisiert. Man sieht, wo der Kurs durchschnittlich lag und wie stark er an welchen Tagen vom Mittelwert abwich. In der Regel verwendet man dafür den Zeitraum der letzten 20 Tage.





> **Bollinger Bands oder Keltner Channels?** Schau hier warum die BB die bessere Wahl wären: https://tradeciety.com/keltner-channel-vs-bollinger-bands-one-better/ Aber statt der Verwendung der Standardabweichung zur Darstellung der Bollinger Bänder wird die Average True Range von Wilders eingesetzt. Dieser Unterschied bewirkt, dass der Keltner Channel ruhiger und gleichmäßiger aussieht als die Bollinger Bänder.



> **EMA**: Exponential Moving Average verwendet man, um Kursschwankungen zu glätten und den zugrunde liegenden Trend dadurch besser zu erkennen. Hierzu betrachtet man einen Kursverlauf für einen bestimmten Zeitraum in der Vergangenheit und berechnet daraus den Durchschnitt. **Formel**: https://admiralmarkets.com/de/wissen/articles/forex-indicators/ema-der-exponentiell-gleitende-durchschnitt-als-universelles-hilfsmittel-fur-ihr-trading




> **Commodity Channel Index (CCI)**: Für den CCI wird die Abweichung zwischen Kurs und dessen Gleitenden Durchschnitt herangezogen, um das Momentum zu erfassen. Um dies zu bewerkstelligen, wird die Abweichung zwischen dem aktuellen Betrachtungszeitraum und der durchschnittlichen Abweichung der vergangenen X Zeitperioden verglichen. Im Gegensatz zum Stochastik-Oszillator und dem RSI sind die Werte des CCI nicht begrenzt. Aber in der Praxis bewegt sich der CCI aufgrund seiner Konstruktion zwischen -100 und +100. 
Bemerkenswerterweise werden beim CCI nicht die Schlusskurse, sondern die typischen Kurse (Hoch + Tief + Schlusskurs / 3) zugrunde gelegt. Dann wird die Abweichung bzw. Entfernung zwischen typischem Kurs und Gleitendem Durchschnitt ermittelt. **Formel**: https://www.kagels-trading.de/technische-indikatoren-trader/ **WICHTIG**: Wie der Name schon sagt, gründet der CCI auf der Annahme, dass sich die Kurse normalerweise innerhalb eines unsichtbaren Kanals um den Gleitenden Durchschnitt herum aufhalten.














# **Indikatoren mit denen wir anfangen sollten**


> 1. EMA


> 2. MACD


> 3. Bollinger Bänder



> 4. RSI












In [3]:
#indicator = pd.DataFrame(df, index=df.index)

In [4]:
def calc_indicators(df):
  ema_25 = 25
  ema_100 = 100
  sma_25 = 25
  sma_100 = 100
  ema_short = 12
  ema_long = 26
  bollinger_avg = 20
  signalline_span = 9

  df['ema_25'] = df['Adj Close'].ewm(span=ema_25, adjust=False).mean()
  df['ema_100'] = df['Adj Close'].ewm(span=ema_100, adjust=False).mean()

  df['sma_25'] = df['Adj Close'].rolling(window=sma_25).mean()
  df['sma_100'] = df['Adj Close'].rolling(window=sma_100).mean()

  df['short_avg_ma'] = df['Adj Close'].rolling(window=bollinger_avg).mean()
  df['short_avg_std'] = df['Adj Close'].rolling(window=bollinger_avg).std()

  df['bollinger'] = df['Adj Close'].ewm(span=bollinger_avg, adjust=False).mean()
  df['bollinger_lower_band'] = df['short_avg_ma'] - (df['short_avg_std'] *2)
  df['bollinger_upper_band'] = df['short_avg_ma'] + (df['short_avg_std'] *2)

  df['ema_short'] = df['Adj Close'].ewm(span=ema_short, adjust=False).mean()
  df['ema_long'] = df['Adj Close'].ewm(span=ema_long, adjust=False).mean()
  df['macd'] = df['ema_short'] - df['ema_long']
  df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

  df['RSI'] = calc_rsi(df)

  df['CCI'] = calc_cci(df)

  # df['OBV_MA'] = calc_obv(df)
   
  df = calculate_dead_cat_bounce_v2(df)

  return df

In [5]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # RolUp = pd.rolling_mean(dUp, n)
    # RolDown = pd.rolling_mean(dDown, n).abs()

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [6]:
def calc_cci(df, n=14, constant=0.015):
    tippingPrice = (df['High'] + df['Low'] + df['Adj Close']) / 3
    tippingPrice_mean = tippingPrice.rolling(n).mean()
    tippingPrice_std = tippingPrice.rolling(n).std()
    
    CCI = pd.Series((tippingPrice - tippingPrice_mean) / (constant * tippingPrice_std)) 
    return CCI

In [7]:
### To Be Adjusted

def calc_obv(df, n = 20):
    i = 0
    OBV = [0]

    while i < df.index[-1]:
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] > 0:
            OBV.append(df.loc[i + 1, 'Volume'])
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] == 0:
            OBV.append(0)
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] < 0:
            OBV.append(-df.loc[i + 1, 'Volume'])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean())

    return OBV_ma

In [8]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Adj Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [72]:
def combine_stocks_to_dataframe(id_list):
  start = dt.datetime(2017,1,1)
  # end = dt.datetime.now()
  end = dt.datetime(2018,1,1)
  df_list = []

  for id in id_list:
    stock = web.DataReader(id, 'yahoo', start, end) # load data
    stock['stock_id'] = id
    stock = calc_indicators(stock) # calc indicators for every stock
    stock.dropna(inplace=True) # maybe delete this later
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [75]:
df_complete = combine_stocks_to_dataframe(['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'SQ', 'V', 'YNDX'])

In [76]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

In [12]:
# def compute_score(row):
#   score=0
#   if row['ema_25']>row['ema_100']:
#     score+=1
#     if row['macd']>row['signalline']:
#       score+=1
#       if row['Adj Close']>row['bollinger_upper_band']:
#         return score
#       else:
#         if ((row['RSI']>=50) & (row['RSI']<=70)) | (row['RSI'] <= 30):
#           score+=1
#         if row['Adj Close']<row['bollinger_lower_band']:
#           score+=1
#   return score

In [13]:
def compute_score_v2(row):
  score=0
  if row['ema_25']>row['ema_100']:
    score+=1
  if row['macd']>row['signalline']:
    score+=1
  if row['Adj Close']>row['bollinger_upper_band']:
    score-=1
  if ((row['RSI']>=50) & (row['RSI']<=70)) | (row['RSI'] <= 30):
    score+=1
  if row['Adj Close']<row['bollinger_lower_band']:
    score+=1
  if row['CCI']<-100:
      score+=1
  if row['Adj Close']>row['sma_25']:
      score+=1
  return score

In [48]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    # if score < 0:
    #     score=0
    return score

def macd_signalline_score(row): # need to change something here
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    adj_close = row['Adj Close']
    score = (macd/adj_close+1)-(signalline/adj_close+1)
    return score

def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>middle_band):
        if adj_close>upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score

In [56]:
def relative_score(row):
    r = row
    stock = row['stock_id']
    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    score = ema_scr+(macd_signalline_scr*10)+bollinger_scr+sma_scr
    print(stock, ema_scr, (macd_signalline_scr*10), bollinger_scr, sma_scr, score, '\n')
    return score

In [77]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd.sort_values(by=['score'], ascending=False)

AMZN 0.07837571382446895 -0.02771685537459545 0.011772585394053792 -0.0018275864201630077 0.060603857423764285 

AAPL 0.04650505295741669 -0.03688653011419074 -0.015542462387779454 -0.016399818674550315 -0.022323758219103818 

SAP 0.020601354891137236 -0.011880158438996036 -0.006710540392337316 -0.00788323678852687 -0.005872580728722987 

ACN 0.06592582465684105 -0.007117845356183761 0.0032370036173559047 0.017412024912013768 0.07945700783002696 

VOW.DE 0.07628733162791002 -0.03727276593438189 -0.003406460585027582 -0.007168162941870504 0.02843994216663004 

PLUG 0.004589071591192706 0.03706592021651134 -0.01937209963204145 -0.010565201605497898 0.011717690570164696 

MDO.DE 0.034850861711375636 -0.006116086798280129 0.002977451501537609 0.005964442179543505 0.03767666859417662 

PYPL 0.07946790367282297 -0.016348066699276576 -0.007533245616143769 -0.010313565579818884 0.04527302577758374 

SQ 0.10139929780752066 -0.0906500383571085 -0.058334007177892944 -0.08420945584883788 -0.131794

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,...,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,,
2017-12-29,154.580002,153.070007,154.429993,153.089996,1452100.0,145.209702,ACN,142.827628,133.993965,142.724578,...,138.959076,147.838879,144.605054,142.688274,1.916780,2.020138,60.968408,46.325067,False,0.079457
2017-12-29,114.889999,114.019997,114.750000,114.019997,5249000.0,111.519279,V,109.897750,105.264245,109.731769,...,106.157684,113.384299,110.650404,109.843486,0.806917,0.715652,61.420538,107.921476,False,0.070802
2017-12-29,1184.000000,1167.500000,1182.349976,1169.469971,2688400.0,1169.469971,AMZN,1163.201569,1078.660762,1171.611191,...,1139.658422,1198.024561,1174.991593,1161.975550,13.016043,16.257446,49.934652,-17.384584,False,0.060604
2017-12-29,74.529999,73.580002,73.919998,73.620003,4496600.0,73.620003,PYPL,74.086555,68.632476,74.387200,...,71.477858,76.172142,74.169531,74.058110,0.111421,0.231775,48.904369,-36.467425,False,0.045273
2017-12-29,33.090000,32.549999,32.570000,32.750000,1587800.0,32.750000,YNDX,32.497985,31.695454,32.723600,...,31.482747,33.328253,32.370511,32.504038,-0.133527,-0.164631,53.782919,54.065213,False,0.039965
2017-12-18,143.399994,143.399994,143.399994,143.399994,0.0,130.878632,MDO.DE,130.182689,125.798503,130.102642,...,128.596341,132.121409,130.708879,130.130915,0.577964,0.658010,80.685988,-17.817416,False,0.037677
2017-12-29,170.699997,168.300003,170.449997,168.699997,46334.0,154.476273,VOW.DE,154.369032,143.427342,155.591578,...,152.357751,159.035094,155.598907,154.225106,1.373801,1.949577,49.223987,-78.020679,False,0.028440
2017-12-29,2.440000,2.350000,2.430000,2.360000,3770800.0,2.360000,PLUG,2.415328,2.404294,2.385200,...,2.304272,2.485728,2.400239,2.417231,-0.016992,-0.025740,43.209027,-45.748591,False,0.011718
2017-12-29,112.690002,111.989998,112.220001,112.360001,287800.0,105.866539,SAP,106.603905,104.452052,106.707741,...,104.150238,108.537294,106.462288,106.603739,-0.141451,-0.015680,45.250577,-50.537407,False,-0.005873


In [47]:

ema_short = 181
ema_long = 166
macd = ema_short-ema_long
signalline = 16
adj_close = 180
# macd = -15
# signalline = 10
# adj_close = 100

(macd/adj_close)-(signalline/adj_close)

-0.005555555555555564

In [18]:
# MACD, RSI, EMA, SMA
# WHAT TO INCLUDE: OBV

def compute_trend_indicators(row):
    score = 0


In [19]:
#print(df_yd.apply(lambda row: compute_score(row), axis=1).sort_values())

# df_complete = combine_stocks_to_dataframe(['AMZN', 'V', 'AAPL', 'VOW.DE', 'PLUG'])

In [20]:
df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)
df_yd

<ipython-input-20-a76c1241de50>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)


,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,...,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-13,3203.840088,3133.000000,3185.469971,3161.469971,3350900.0,3161.469971,AMZN,3283.549082,3220.995443,3340.004395,...,3138.627442,3525.015551,3264.297194,3283.014033,-18.716839,15.786784,32.502502,-94.748667,False,1
2021-05-13,126.150002,124.260002,124.580002,124.970001,105752400.0,124.970001,AAPL,128.837766,126.478966,131.165659,...,123.573002,137.933942,127.949043,128.836420,-0.887378,0.286905,32.493133,-84.298176,False,1
2021-05-13,136.110001,134.570007,135.559998,135.229996,611000.0,135.229996,SAP,137.510924,132.570564,139.374799,...,134.840900,146.017101,138.565622,137.372665,1.192956,2.430710,33.015917,-111.932450,False,2
2021-05-13,287.250000,281.119995,281.119995,285.320007,1698200.0,285.320007,ACN,286.110886,268.264198,288.557008,...,283.261225,295.390777,287.816380,285.869844,1.946537,3.664398,41.804382,-103.612118,False,2
2021-05-14,262.600006,258.399994,259.200012,262.600006,19090.0,262.600006,VOW.DE,268.902867,236.481487,273.816001,...,247.920258,291.959744,263.997207,268.948814,-4.951607,-3.346059,40.362248,-32.411224,False,1
2021-05-13,23.610001,21.299999,22.740000,21.959999,38327700.0,21.959999,PLUG,26.812307,34.285883,26.303200,...,20.182527,31.000472,24.035306,27.033710,-2.998404,-2.998904,34.776036,-65.024117,False,1
2021-05-14,191.050003,191.050003,191.050003,191.050003,100.0,191.050003,MDO.DE,192.470335,185.071788,193.568000,...,190.566033,196.563966,192.806867,192.379713,0.427154,1.204189,40.490659,-81.417448,False,1
2021-05-13,245.679993,237.800003,241.919998,240.800003,7268100.0,240.800003,PYPL,254.737726,247.019398,260.983200,...,237.353639,279.860364,250.729693,254.849121,-4.119428,-1.236908,31.158254,-87.852710,False,1
2021-05-13,211.500000,192.289993,208.199997,197.130005,21371200.0,197.130005,SQ,232.918327,227.873015,242.662000,...,203.773717,270.725282,224.499018,233.211743,-8.712725,-3.413529,22.939908,-128.081533,False,4


In [21]:
df_complete[df_complete.dead_cat_bounce==True]

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,...,bollinger,bollinger_lower_band,bollinger_upper_band,ema_short,ema_long,macd,signalline,RSI,CCI,dead_cat_bounce
Date,,,,,,,,,,,,,,,,,,,,,
2020-10-28,110.339996,107.699997,109.669998,108.110001,5486700.0,108.110001,SAP,145.454121,148.740207,149.296800,...,143.028637,116.717681,179.038320,136.493572,145.849917,-9.356345,-4.462065,9.557939,-137.607496,True
2020-10-29,109.870003,106.059998,106.800003,108.629997,2824800.0,108.629997,SAP,142.621496,147.945945,147.508400,...,139.752576,110.056879,180.880120,132.206868,143.092886,-10.886018,-5.746856,9.844080,-112.666718,True
2020-09-24,220.449997,213.149994,218.869995,214.419998,4828600.0,212.199539,ACN,231.450523,217.639759,234.501379,...,231.527577,221.427159,247.008135,230.264042,231.385243,-1.121200,1.208118,24.030213,-209.301065,True
2020-09-25,215.070007,210.419998,213.029999,214.630005,3691100.0,212.407364,ACN,229.985664,217.536147,233.631283,...,229.706604,217.013155,248.825311,227.516861,229.979474,-2.462613,0.473972,25.204056,-159.812921,True
2020-07-31,140.199997,132.699997,138.000000,132.699997,91147.0,128.430832,VOW.DE,140.943584,141.077473,141.415214,...,140.850220,133.049339,150.640522,139.987165,140.940759,-0.953594,0.590261,22.148804,-173.978706,True
2020-10-28,136.800003,131.800003,136.300003,133.500000,142287.0,133.500000,VOW.DE,144.791133,144.375611,145.078564,...,144.445724,138.177402,152.812600,143.312620,144.841505,-1.528885,-0.338882,23.690957,-193.654246,True
2021-02-18,54.480000,50.060001,51.880001,50.230000,34222100.0,50.230000,PLUG,59.380188,39.722116,64.281600,...,60.514456,54.706293,73.482707,60.716823,59.106783,1.610041,4.333278,21.347698,-171.602557,True
2021-03-25,34.180000,30.100000,31.000000,34.099998,35469800.0,34.099998,PLUG,43.305191,40.869021,43.284000,...,41.900997,31.200699,52.552301,39.032677,43.541913,-4.509236,-3.954665,36.760703,-107.943324,True
2021-04-15,28.770000,24.610001,28.750000,26.090000,69174300.0,26.090000,PLUG,35.218989,38.667813,35.596400,...,33.773345,26.706104,39.896896,31.315878,35.491190,-4.175311,-4.009442,26.838968,-140.075752,True


In [22]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()

In [23]:
dead_cat_bounce = calculate_dead_cat_bounce()
#bot_warn_me('Watch Out! Check Code')
dead_cat_bounce

NameError: name 'calculate_dead_cat_bounce' is not defined